In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create the model
model = Network('model')

In [3]:
# Define network nodes
source = Source(model, 'Source')
queue = Queue(model, 'Queue1', SchedStrategy.FCFS)
sink = Sink(model, 'Sink')
vsink1 = Router(model, 'VSink1')  # Virtual sink 1
vsink2 = Router(model, 'VSink2')  # Virtual sink 2

In [4]:
# Define job classes
ocl1 = OpenClass(model, 'Class1')
ocl2 = OpenClass(model, 'Class2')

In [5]:
# Set arrival processes
source.set_arrival(ocl1, Exp(1.0))  # Class1: arrival rate 1.0
source.set_arrival(ocl2, Exp(1.0))  # Class2: arrival rate 1.0

In [6]:
# Set service processes
queue.set_service(ocl1, Exp(100.0))  # Class1: very fast service (rate 100)
queue.set_service(ocl2, Exp(100.0))  # Class2: very fast service (rate 100)

In [7]:
# Configure routing matrix with virtual sinks
P = model.init_routing_matrix()

# Class1 routing
P.set(ocl1, ocl1, source, queue, 1.0)     # Source → Queue1
P.set(ocl1, ocl1, queue, vsink1, 0.6)     # Queue1 → VSink1 (60%)
P.set(ocl1, ocl1, queue, vsink2, 0.4)     # Queue1 → VSink2 (40%)
P.set(ocl1, ocl1, vsink1, sink, 1.0)      # VSink1 → Sink
P.set(ocl1, ocl1, vsink2, sink, 1.0)      # VSink2 → Sink

# Class2 routing
P.set(ocl2, ocl2, source, queue, 1.0)     # Source → Queue1
P.set(ocl2, ocl2, queue, vsink1, 0.1)     # Queue1 → VSink1 (10%)
P.set(ocl2, ocl2, queue, vsink2, 0.9)     # Queue1 → VSink2 (90%)
P.set(ocl2, ocl2, vsink1, sink, 1.0)      # VSink1 → Sink
P.set(ocl2, ocl2, vsink2, sink, 1.0)      # VSink2 → Sink

model.link(P)

In [8]:
# Expected throughput calculations
# Input rates
class1_arrival = 1.0
class2_arrival = 1.0

# Class1 flows
class1_to_vsink1 = class1_arrival * 0.6
class1_to_vsink2 = class1_arrival * 0.4

# Class2 flows
class2_to_vsink1 = class2_arrival * 0.1
class2_to_vsink2 = class2_arrival * 0.9

# Total flows to each virtual sink
total_to_vsink1 = class1_to_vsink1 + class2_to_vsink1
total_to_vsink2 = class1_to_vsink2 + class2_to_vsink2

print(f"Class1 flows:")
print(f"  To VSink1: {class1_to_vsink1:.1f}")
print(f"  To VSink2: {class1_to_vsink2:.1f}")

print(f"\nClass2 flows:")
print(f"  To VSink1: {class2_to_vsink1:.1f}")
print(f"  To VSink2: {class2_to_vsink2:.1f}")

Class1 flows:
  To VSink1: 0.6
  To VSink2: 0.4

Class2 flows:
  To VSink1: 0.1
  To VSink2: 0.9


In [ ]:
# Aligned with JAR test scenarios for oqn_vsinks
# JAR tests: MVA(), MAM(), NC()

# Solve with MVA solver (matches JAR)
solver_mva = SolverMVA(model)
AvgTable = solver_mva.get_avg_table()

In [10]:
AvgNodeTable = solver_mva.getAvgNodeTable()

     Node JobClass    QLen  Util   RespT  ResidT  ArvR  Tput
0  Source   Class1  0.0000  0.00  0.0000  0.0000   0.0   1.0
1  Source   Class2  0.0000  0.00  0.0000  0.0000   0.0   1.0
2  Queue1   Class1  0.0102  0.01  0.0102  0.0102   1.0   1.0
3  Queue1   Class2  0.0102  0.01  0.0102  0.0102   1.0   1.0
4    Sink   Class1  0.0000  0.00  0.0000  0.0000   1.0   0.0
5    Sink   Class2  0.0000  0.00  0.0000  0.0000   1.0   0.0
6  VSink1   Class1  0.0000  0.00  0.0000  0.0000   0.6   0.6
7  VSink1   Class2  0.0000  0.00  0.0000  0.0000   0.1   0.1
8  VSink2   Class1  0.0000  0.00  0.0000  0.0000   0.4   0.4
9  VSink2   Class2  0.0000  0.00  0.0000  0.0000   0.9   0.9


In [ ]:
# Solve with MAM solver for comparison (matches JAR)
solver_mam = SolverMAM(model)
AvgTableMAM = solver_mam.get_avg_table()

In [12]:
AvgNodeTableMAM = solver_mam.getAvgNodeTable()

     Node JobClass    QLen  Util   RespT  ResidT  ArvR  Tput
0  Source   Class1  0.0000  0.00  0.0000  0.0000   0.0   1.0
1  Source   Class2  0.0000  0.00  0.0000  0.0000   0.0   1.0
2  Queue1   Class1  0.0102  0.01  0.0102  0.0102   1.0   1.0
3  Queue1   Class2  0.0102  0.01  0.0102  0.0102   1.0   1.0
4    Sink   Class1  0.0000  0.00  0.0000  0.0000   1.0   0.0
5    Sink   Class2  0.0000  0.00  0.0000  0.0000   1.0   0.0
6  VSink1   Class1  0.0000  0.00  0.0000  0.0000   0.6   0.6
7  VSink1   Class2  0.0000  0.00  0.0000  0.0000   0.1   0.1
8  VSink2   Class1  0.0000  0.00  0.0000  0.0000   0.4   0.4
9  VSink2   Class2  0.0000  0.00  0.0000  0.0000   0.9   0.9


In [ ]:
# Solve with NC solver for comparison (matches JAR)
solver_nc = SolverNC(model)
AvgTableNC = solver_nc.get_avg_table()

In [14]:
AvgNodeTableNC = solver_nc.getAvgNodeTable()

     Node JobClass    QLen  Util   RespT  ResidT  ArvR  Tput
0  Source   Class1  0.0000  0.00  0.0000  0.0000   0.0   1.0
1  Source   Class2  0.0000  0.00  0.0000  0.0000   0.0   1.0
2  Queue1   Class1  0.0102  0.01  0.0102  0.0102   1.0   1.0
3  Queue1   Class2  0.0102  0.01  0.0102  0.0102   1.0   1.0
4    Sink   Class1  0.0000  0.00  0.0000  0.0000   1.0   0.0
5    Sink   Class2  0.0000  0.00  0.0000  0.0000   1.0   0.0
6  VSink1   Class1  0.0000  0.00  0.0000  0.0000   0.6   0.6
7  VSink1   Class2  0.0000  0.00  0.0000  0.0000   0.1   0.1
8  VSink2   Class1  0.0000  0.00  0.0000  0.0000   0.4   0.4
9  VSink2   Class2  0.0000  0.00  0.0000  0.0000   0.9   0.9
